# Objective: Issue Verification Codes to patients stored as REDCap records

Case Investigation and Contact Tracing (CICT) migration to REDCap will require an issue of Exposure Notification codes to patients that
have provided a phone number in the REDCap records.

#### Meeting with Daniel L. 02/23/2022
 - server may issue 500 code during server maintenance, response may return as text instead of json. Implement retry logic with exponential back-off algorithm. 
 - 400 errors, bad phone number, or date out of range, code will not be issued. Need logic to record case outcome (in redcap?)
 - padding, create a function to randomly generate string of length 1-2624, input as padding. 
 - generate UUID (UNiversal Unique Identifier):
     - use MD5 function (pre-made in python, numpy?)
     - two arguments as input to MD5: patient phone number, unique argument to me (I generate this). 
     - Out put of MD5 is my UUID, include that to send to server. 
     - server will store UUID, this will be used to prevent repeat code issuing in the future. 

#### Questions:
    - difference between verification code and long term verification token?
    - what is verification certificate?

In [1]:
import requests
import json
import string
import random
import hashlib

from config import api_key
from config import uuid_salt

# Single code issue example:

In [ ]:
hed = {
    'x-api-key': api_key,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
    }
url = "https://adminapi.encv-test.org/api/issue"

data = json.dumps({
    "symptomDate": "2022-02-06",
    "testDate": "2022-02-07",
    "testType": "confirmed",
    "tzOffset": -480,
    "phone": "+1 971-222-7382",
    "padding": "U11krYIjr8eX"
    })

In [ ]:
response = requests.post(url, headers=hed, data=data)
print(response.status_code)

In [ ]:
print(response.request.headers)
print(response.request.body)
print(response.request.url)

In [ ]:
response.text

# Batch Issue example

In [ ]:
hed = {
    'x-api-key': api_key,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

url = "https://adminapi.encv-test.org/api/batch-issue"

data = json.dumps({
    "codes": [
        {
            "symptomDate": "2022-02-06",
            "testDate": "2022-02-07",
            "testType": "confirmed",
            "tzOffset": -480,
            "phone": "+1 971-222-7382",
            
        },
        {
            "symptomDate": "2022-02-06",
            "testDate": "2022-02-07",
            "testType": "confirmed",
            "tzOffset": -480,
            "phone": "+1 971-222-7383",
        }
    ],
    "padding": "U11krYIjr8eX"
})

In [ ]:
response = requests.post(url, headers=hed, data=data)
print(response.status_code)

In [ ]:
response.request.body

In [ ]:
response.content

### Generate Padding Function

In [ ]:
def padding_gen():
    """
    Generate a string of random length between 0 and 3000 characters
    to be used as padding for API calls
    
    return:
        string 
    """
    
    chars = string.ascii_uppercase + string.digits + string.ascii_lowercase
    length = random.randint(0,3000)
    
    return "".join([random.choice(chars) for i in range(length)])

### Generate UUID per phone number

In [2]:
def uuid_gen(text, salt = uuid_salt):
    """
    Generate a uuid from phone number string and HMAC key (unique passphrase)
    
    return:
        string
    """
    textutf8 = text.encode("utf-8")
    saltutf8 = uuid_salt.encode("utf-8")
    
    hash_key = hashlib.md5(saltutf8+b":"+textutf8)
    hexa = hash_key.hexdigest()
    
    return hexa
    